In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. 데이터 로더 설정
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # CIFAR-10 정규화
])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=False, # 순차적으로 데이터 로드
    num_workers=0  # ★ 변경: 2 -> 0 Windows에서 __main__ 가드 없이도 에러 안 나도록
)

# 2. 모델 정의
class CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CIFAR10Classifier()

# 3. 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss() # 다중 클래스 분류를 위한 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # 경사 하강법

# 4. 모델 학습
epochs = 10 # 학습 반복 수
running_loss = 0.0
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # 순전파: 입력 데이터를 모델에 전달해 출력 계산
        outputs = model(inputs)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파: 기울기 계산 및 가중치 업데이트
        optimizer.zero_grad() # 이전 기울기 초기화
        loss.backward() # 역전파로 기울기 계산
        optimizer.step() # 옵티마이저로 가중치 업데이트

        # 손실 누적
        running_loss += loss.item()
        if (i + 1) % 100 == 0: # 100번째 배치마다 손실 출력
            print(f'Epoch [{epoch+1}/{epochs}], \
            Step [{i+1}/{len(train_loader)}], \
            Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('학습 완료!')

# 모델 평가 함수
def evaluate_model(model, train_loader):
    model.eval() # 평가 모드 (dropout, batchnorm 비활성화)
    correct = 0 # 맞춘 샘플 개수
    total = 0 # 전체 샘플 개수

    with torch.no_grad(): # 평가 중에는 기울기 계산하지 않음
        for inputs, labels in train_loader:
            outputs = model(inputs) # 모델 예측
            _, predicted = torch.max(outputs.data, 1) # 가장 높은 점수의 클래스 선택
            total += labels.size(0) # 전체 샘플 개수 업데이트
            correct += (predicted == labels).sum().item() # 맞춘 샘플 개수 업데이트

    accuracy = correct / total * 100 # 정확도 계산 (백분율)
    print(f'테스트 데이터셋 정확도: {accuracy:.2f}%')

# 모델 평가
evaluate_model(model, train_loader)

100%|██████████| 170M/170M [00:01<00:00, 98.5MB/s]


Epoch [1/10],             Step [100/157],             Loss: 2.1761
Epoch [2/10],             Step [100/157],             Loss: 2.7765
Epoch [3/10],             Step [100/157],             Loss: 2.3854
Epoch [4/10],             Step [100/157],             Loss: 2.1608
Epoch [5/10],             Step [100/157],             Loss: 1.9801
Epoch [6/10],             Step [100/157],             Loss: 1.8165
Epoch [7/10],             Step [100/157],             Loss: 1.6652
Epoch [8/10],             Step [100/157],             Loss: 1.5074
Epoch [9/10],             Step [100/157],             Loss: 1.3336
Epoch [10/10],             Step [100/157],             Loss: 1.1748
학습 완료!
테스트 데이터셋 정확도: 74.18%
